# Winning probabilities at the 6/49 lottery game

Lotto 6/49, launched on June 12, 1982, was the first Canadian Lottery game to allow players to choose their own numbers. 
Winning numbers are drawn every Wednesday and Saturday by the Interprovincial Lottery Corporation. 
Each time, 6 numbers are drawn wthout replacement from a set of 49, plus a bonus number. 
A player wins money if at least two of the first six are among the numbers they chose. 
To win the hoghest amount, called the ‘jackpot prize’, a player must have the first six correct numbers. 

Although most people start playing the lottery for fun, it can also turn into an addiction or some players, with potentially disastrous consequences. 
One important factor in the development of such addictions is the communication centred on the large amount of money the jackpot prize represents and on the change of life it can entail. 
This focus on a technically possible but unlikely outcome can overshadow how small its probability is, or, more prosaically, that a player is unlikely to win anything at all. 

In this short project, I will estimate some winning likelihoods, in the hope that realizing their smallness may help prevent addiction. 
I shall mostly use basic Python, in an effort to make the code as transparent as possible, and since the relatively small numbers involved do not require a lot of computing power. 
I recommend the use of a dedicated library, *e.g.* SciPy, if using the functions defined here on very large integers.

## Basic combinatorial functions

Let us first define two functions: 
* `factorial`, taking a non-negative integer $n$ and returning $n!$, 
* `combination`, taking two non-negative integers $n$ and $k$ and returning ${n\choose k}$.

In [1]:
def factorial(n):
    '''
    returns the factorial of n 
    n: non-negative integer
    '''
    res = 1
    for i in range(1, n+1):
        res *= i
    return res

def combination(n,k):
    '''
    returns the number of combinations of k elements among n
    n and k: non-negative integers 
    '''
    if k > n:
        return 0
    else:
        return factorial(n) / (factorial(n-k) * factorial(k))

Let us also define the function `approximate_fraction_num` taking a float and returning a fraction close to it. 
It takes an optional parameter `max_num`, equal to 10 by default, representing the maximum acceptable value for the numerator.

In [2]:
def gcd(a, b):
    '''
    returns the greatest common divisor to a and b
    a: positive integer
    b: positive integer
    '''
    a_ = max(a,b)
    b_ = min(a,b)
    if b_ == 0:
        return a_
    r_ = a_ % b_
    while(r_ != 0):
        a_ = b_
        b_ = r_
        r_ = a_ % b_
    return b_

def approximate_fraction(f, max_den = 20):
    '''
    returns the numerator and denominator of a fraction approximating f, with 
    a maximum denominator max_den
    f: float
    max_den: positive integer
    '''
    if f == 0.:
        return (0,1)
    num = int(f * max_den)
    if abs(f) * max_den - abs(num) > 0.5: 
        num += (f / abs(f))*1
    r = gcd(abs(num), max_den)
    num = int(num / r)
    den = int(max_den / r)
    return (num, den)

def approximate_fraction_num(f, max_num = 10):
    '''
    returns the numerator and denominator of a fraction approximating f, with 
    a maximum numerator max_num
    f: float
    max_num: positive integer
    '''
    frac_inv = approximate_fraction(1./f, max_den = max_num)
    return (frac_inv[1], frac_inv[0])

## Some winning probabilities

### Winning the jackpot prize in one try

Let $n$ be the number of elements in the set and $k$ be the number of elements drawn (in the case of the 6/49 game, they are respectively 49 and 6). 
The probability $P_1(J)$ of winning the jackpot prize by playing once (*i.e.*, choosing a set of six numbers) is: 
$$ P_1(J) = {n \choose k}^{-1} .$$
Let us compute this number for $n = 49$ and $k = 6$: 

In [3]:
n = 49
k = 6
P_1_J = 1. / combination(n,k)
frac = approximate_fraction_num(P_1_J)
print('Winning probability: {:.1e}, i.e., {:.7f}, or {} over {}'.format(P_1_J, P_1_J*100, frac[0], frac[1]))

Winning probability: 7.2e-08, i.e., 0.0000072, or 1 over 13983816


The probability of winning the jackpot prize is smaller than one over ten millions, which is rather small.

### Would I have won? 

To get more intuition about how small this number is, we want to write a function taking a six-numbers input and telling if, assuming this set was played many times, it would have won the jackpot prize at least once. 
To this end, we first load the dataset `649.csv`, containing results from previous draws.

In [4]:
import pandas as pd
df = pd.read_csv('../Data/649.csv')
print(df.shape)
display(df.head())
display(df.tail())

(3665, 11)


,PRODUCT,DRAW NUMBER,SEQUENCE NUMBER,DRAW DATE,NUMBER DRAWN 1,NUMBER DRAWN 2,NUMBER DRAWN 3,NUMBER DRAWN 4,NUMBER DRAWN 5,NUMBER DRAWN 6,BONUS NUMBER
0,649,1,0,6/12/1982,3,11,12,14,41,43,13
1,649,2,0,6/19/1982,8,33,36,37,39,41,9
2,649,3,0,6/26/1982,1,6,23,24,27,39,34
3,649,4,0,7/3/1982,3,9,10,13,20,43,34
4,649,5,0,7/10/1982,5,14,21,31,34,47,45


,PRODUCT,DRAW NUMBER,SEQUENCE NUMBER,DRAW DATE,NUMBER DRAWN 1,NUMBER DRAWN 2,NUMBER DRAWN 3,NUMBER DRAWN 4,NUMBER DRAWN 5,NUMBER DRAWN 6,BONUS NUMBER
3660,649,3587,0,6/6/2018,10,15,23,38,40,41,35
3661,649,3588,0,6/9/2018,19,25,31,36,46,47,26
3662,649,3589,0,6/13/2018,6,22,24,31,32,34,16
3663,649,3590,0,6/16/2018,2,15,21,31,38,49,8
3664,649,3591,0,6/20/2018,14,24,31,35,37,48,17


This dataset contains results from all drawings that occurred before 21st June 2018. 
Let us use it to build the list `drawings_6` of all drawings, excluding the bonus number. 
We cast them as sets since the order is not important.

In [5]:
df['drawing_6'] = df[['NUMBER DRAWN ' + str(i) for i in range(1,7)]].values.tolist()
drawing_6 = list(df['drawing_6'].apply(set))

We now define the function `has_aready_won` taking a six-numbers list or set and returning `True` if it has been drawn at least once and `False` otherwise (technically, it returns whether it has been drawn before June 21st 2018; but this only make a difference for a small number of sets):

In [6]:
def has_already_won(numbers):
    return set(numbers) in drawing_6

Let us test it on two examples: one which was drawn and one which was not.

In [7]:
print(has_already_won([3,11,12,14,43,41]))
print(has_already_won([3,11,13,14,43,41]))

True
False


Let us now do a small experiment: we select a large number (say, 10000) of (not necessarily different) sets of six numbers at random, and count how much of them have been drawn at least once: 

In [8]:
from numpy.random import choice, seed

n_sets = 10000

seed(1) # fix the seed to get the same result each time the code is run
n_has_won = 0
for i in range(n_sets): 
    test_set = choice(range(1,50), size=6, replace=False)
    if has_already_won(test_set): 
        n_has_won += 1

print('Among the {} sets, {} have been drawn at least once'.format(n_sets, n_has_won))

Among the 10000 sets, 2 have been drawn at least once


Only 2 sets among the 1000 have been drawn at least once. 
The probability for a given set to be drawn one specific day is thus quite small!

### Winning the jackpot price with several tickets

Assuming we play $l$ different valid tickets, where $l$ is a positive integer, the probability $P_l(J)$ to win the jackpot prize is:
$$P_l(J) = l \times P_1(J).$$
From the above results, having a probability of just 1 in 10 of winning would require playing more than one million sets of numbers. 
At the time of writing (21 April 2020), the cost of playing six numbers is CAD3. 
Having a probability to win the jackpot prize of more than one half requires buying more than 5,000,000 of them, for a total cost of CAD15,000,000. 
Since the jackpot prize is only CAD6,000,000 at the moment, this would incur a large loss of money even if winning.

### Having a few matching numbers

We now focus on smaller prizes. 
The function `proba_prize` below takes 4 inputs: 
* n (positive integer): the number of elements in the set, 
* k (positive integer): the number of elements to be chosen by the player, 
* l (positive integer): the number of elements that need to match for a given prize, excluding the bonus number, 
* b (boolean): whether the bonus number needs to match

It returns the probability of winning the price when playing once. 
All inputs defaut to the jackpot prize setup.

In [9]:
def proba_prize(n=49, k=6, l=6, b=False):
    proba_l_numbers = 0.
    # compute the probability that at least l numbers match
    for l_ in range(l, k+1):
        # probability that exactly l_ elements match
        # for each choice of l_ elements among the k, the number of draws where 
        # they match while the k-l_ others don't match is equal to the number of
        # ways of choosing k-l_ elements among n-k
        proba_l_numbers += combination(k,l_) * combination(n-k,k-l_) / combination(n,k)
    if b:
        # if the bonus number needs to match
        return proba_l_numbers / n
    else:
        return proba_l_numbers

Let us apply this function to $l \in \lbrace 2, 3, 4, 5, 6 \rbrace$: 

In [10]:
for l in range(2, 7):
    proba = proba_prize(l=l)
    frac = approximate_fraction_num(proba)
    if frac[0] == 1:
        print('Probability of having at least {} matching numbers: {:.2e}, i.e., approximately one in {}'.format(l, proba, int(1./proba)))
    else:
        print('Probability of having at least {} matching numbers: {:.2e}, i.e., approximately {} in {}, or one in {}'.format(l, proba, frac[0], frac[1], int(1./proba)))

Probability of having at least 2 matching numbers: 1.51e-01, i.e., approximately 5 in 33, or one in 6
Probability of having at least 3 matching numbers: 1.86e-02, i.e., approximately 10 in 537, or one in 53
Probability of having at least 4 matching numbers: 9.87e-04, i.e., approximately one in 1013
Probability of having at least 5 matching numbers: 1.85e-05, i.e., approximately 5 in 269958, or one in 53991
Probability of having at least 6 matching numbers: 7.15e-08, i.e., approximately one in 13983816


Let us also compute the probability of winning the second prize, i.e., having 5 matching numbers plus the bonus number:

In [11]:
proba = proba_prize(l=5, b=True)
print('Probability of winning the second prize: {:.2e}, i.e., approximately one in {}'.format(proba, int(1./proba)))

Probability of winning the second prize: 3.78e-07, i.e., approximately one in 2645586


All these probabilities are rather small. 
Even the probability of having just two matching numbers, which, under the current rules (as of 12/02/2020) is the lowest possible prize and gives only a free ticket, is just one in six.

## Average win (or loss)

Finally, let us compute the average gain for each play, using data from [the bclc website](https://lotto.bclc.com/lotto-649-and-extra/prizes-and-odds.html) (valid on 21 April 2020). 
We use that, if $P$ is the probability of winning a prize $p$ and $N$ the number of players, the average number of players winning the prize is $N P$. The average gain for each player is thue $(P p) / (N P) = p / N$.

In [12]:
n = 49
k = 6

# cost of playing in CAD
cost_line = 3

# prize amount in CAD
prop_in_play = 0.47 # proportion of sales put in play

# estimate for the number of players
# The result actually does no depend on this number as, in each term in the 
# sum, it appears both on the numerator and denominator. However, we keep 
# it for clarity. 
number_players = 3000000 
pools_fund = number_players * cost_line * prop_in_play
prize_amount = [0.795*pools_fund, 0.06*pools_fund, 0.05*pools_fund, 0.095*pools_fund, 10., 5., 3.]

average_gain = sum([prize_amount[i] / number_players for i in range(len(prize_amount))]) - cost_line
print('Average gain: {:.2f}'.format(average_gain))

Average gain: -1.59


On average, a player will thus lose more than half the price of a ticket. 
Let us see how much a regular player may lose in 10 years.

In [13]:
# average gain * (two days a week) * (average number of days in a year)
# * (number of years)
average_gain * (2/7) * 365.2475 * 10

-1659.2609529000001

A player playing twice a week for 10 years will, on average, lose more than CAD 1650.